In [1]:
import sys
import pandas as pd
import numpy as np
import altair as alt

from importlib import reload

alt.themes.enable('vox')
sys.path.append('/home/developer/python_libs/')
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

# SoH comparison

In [2]:
apr18650_df = pd.read_csv('plots_data/capacity_fading_apr18650m1.csv')
anr26650_df = pd.read_csv('plots_data/capacity_fading_anr26650m1a.csv')
anr26650_df.rename({'25_deg_x': 'x', '25_deg_y': 'y'}, axis=1, inplace=True)
anr26560_df = anr26650_df.iloc[:, 0:2].dropna()
apr18650_df['battery'] = 'A123 APR18650M1'
anr26650_df['battery'] = 'A123 ANR26650M1A'

apr18650_df['y'] /= 100
anr26650_df['y'] /= 100

In [3]:
alt.Chart(pd.concat([apr18650_df, anr26650_df])).encode(
    x=alt.X('x:Q', title='Cycles (n)'),
    y=alt.Y('y:Q', scale=alt.Scale(domain=[0.8, 1], zero=False), title='SoH (%)'),
    color=alt.Color('battery:N',
                    scale=alt.Scale(
                        domain=['A123 ANR26650M1A', 'A123 APR18650M1'], scheme='set1'),
                    legend=alt.Legend(title='Battery type'),
                    )

).mark_line(interpolate='bundle').transform_filter(alt.datum.x < 1000).configure_axis(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=17,
    titleFontSize=19,
).configure_legend(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=17,
    titleFontSize=19,
).configure_view(
    width=600
)

alt.Chart(...)

# Model parts comparison

In [32]:
import mpc_optimization.mpc_optimizer
import mpc_optimization.battery_model_constants
reload(mpc_optimization.mpc_optimizer)
reload(mpc_optimization.battery_model_constants)

from mpc_optimization.battery_model_constants import Battery_state_vars, State_vars_aliases_dict, C_rate_per_second
from mpc_optimization.fmu_source import FmuSource, ModelicaModelInfo
from mpc_optimization.mpc_optimizer import MPCOptimizer

from pathlib import Path

def get_soc_dependence(variable, is_charging=True):
    mpc_optimizer = MPCOptimizer(
        model_info=ModelicaModelInfo(Path("/home/developer/modelica/BatteryWithFullCycle.mo"), 
                                    "BatteryMPC.BatteryWithFullCycle"),
        fmu_path="/home/developer/ipynotebooks/BatteryMPC.BatteryWithFullCycle.fmu",
        state_variables=Battery_state_vars,
        input_vec=['I_req'],
        output_vec=['SoC',
                    'theveninBasedBattery.SoH'],
        horizon_num=5,
        points_per_sec=.1
    )
    
    start_time = 0
    final_time = 7200

    control_df = pd.DataFrame({"time": range(start_time, final_time)})
    control_df['I_req'] = C_rate_per_second/1.1
    control_df.loc[control_df.index > 3600, 'I_req'] = -C_rate_per_second/1.1
#     control_df.loc[control_df.index >= 6800, 'I_req'] = 0
    control_df.set_index('time', inplace=True)
    
    states = mpc_optimizer.simulate(start_time, final_time, control_df, save_all=True)
    states[State_vars_aliases_dict['U']] *= -1

    states = states.rename(columns={v: k for k, v in State_vars_aliases_dict.items()})
    if variable.startswith('theveninBasedBattery.C'):
        var, soc, t = states[variable]*3600, states['SoC'], states['time']
    elif variable.startswith('theveninBasedBattery.R'):
        var, soc, t = states[variable]/3600, states['SoC'], states['time']
    else:
        var, soc, t = states[variable], states['SoC'], states['time']
    
    st = np.argmin(np.abs(t - 3600)) if is_charging else 0
    en = np.argmin(np.abs(t - 7200)) if is_charging else np.argmin(np.abs(t - 3600))
    simulation_res = pd.DataFrame({"SoC": soc[st:en], variable: var[st:en]})
    simulation_res['type'] = 'simulation'
    
    return simulation_res

def get_param_charts(is_charging):
    charts = dict()
    modifier = '_charging' if is_charging else '_discharging'
    for filename in sorted(Path(".").glob('plots_data/thevenin_params/*' + modifier + '.csv')):
        df = pd.read_csv(filename)
        def remove_modifier(x): return "_".join(x.split('_')[:-1])
        component = remove_modifier(filename.stem)
        component_type = remove_modifier(component)
        variable = ".".join(['theveninBasedBattery', component, component_type])
        sim_df = get_soc_dependence(variable, is_charging).rename({variable: component}, axis=1)
        df.rename(columns={'x': 'SoC', 'y': component}, inplace=True)
        df.sort_values('SoC', inplace=True)
        df['type'] = 'previous work'
        unit = 'Ohm' if component_type == 'R' else 'Farad'
        chrt = alt.Chart(df.append(sim_df)).encode(
            x=alt.X('SoC:Q', title='SoC'),
            y=alt.Y(field=component, type='quantitative', title=component_type + '(' + unit +')', scale=alt.Scale(zero=True)),
            color=alt.Color('type:N', scale=alt.Scale(scheme='set1'))
        ).mark_line(
        ).transform_filter(
            alt.datum.SoC > 0.05
        ).properties(
            title=component
#             title=modifier[1:]
        )


        charts[component] = chrt
    return charts
        

In [8]:
discharging_charts = get_param_charts(not True)
charging_charts = get_param_charts(True)

100%|██████████| 720/720 [00:01<00:00, 495.16it/s]


In [43]:
alt.hconcat(
    alt.vconcat(*charging_charts.values()).properties(title='Charging'),
    alt.vconcat(*discharging_charts.values()).properties(title='Discharging'),
).configure_view(
    height=200,
    width=300
).configure_legend(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=17,
    titleFontSize=19,
).configure_axis(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=17,
    titleFontSize=19,
).configure_title(
    font='CMU Serif',
    fontSize=17,
    anchor='middle'
)

alt.HConcatChart(...)

# Open-circuit voltage

In [39]:
simulation_u_ocv = get_soc_dependence('U_ocv')#.rename({State_vars_aliases_dict['U_ocv']: 'U_ocv'}, axis=1)
# simulation_u_ocv = get_soc_dependence('SoC').rename({'SoC': 'U_ocv'}, axis=1)
paper_u_ocv = pd.read_csv('plots_data/u_ocv.csv').rename({"x": 'SoC', 'y': 'U_ocv'}, axis=1)
paper_u_ocv['type'] = 'previous work'

alt.Chart(simulation_u_ocv.append(paper_u_ocv)).mark_line().encode(
    x = alt.X('SoC:Q'),
    y = alt.Y('U_ocv:Q', scale=alt.Scale(zero=False), title='U_oc(SoC)'),
    color=alt.Color('type:N', scale=alt.Scale(scheme='set1'))
).configure_view(
    height=300,
    width=600
).configure_legend(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=17,
    titleFontSize=19,
).configure_axis(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=17,
    titleFontSize=19,
)

100%|██████████| 720/720 [00:01<00:00, 656.27it/s]


alt.Chart(...)

# Test cycle comparison

In [37]:
import mpc_optimization.mpc_optimizer
import mpc_optimization.battery_model_constants
reload(mpc_optimization.mpc_optimizer)
reload(mpc_optimization.battery_model_constants)

from mpc_optimization.battery_model_constants import Battery_state_vars, State_vars_aliases_dict, C_rate_per_second
from mpc_optimization.fmu_source import FmuSource, ModelicaModelInfo
from mpc_optimization.mpc_optimizer import MPCOptimizer

def get_test_voltage_response(is_charging):
    mpc_optimizer = MPCOptimizer(
        model_info=ModelicaModelInfo(Path("/home/developer/modelica/BatteryWithFullCycle.mo"), 
                                    "BatteryMPC.BatteryWithFullCycle"),
        fmu_path="/home/developer/ipynotebooks/BatteryMPC.BatteryWithFullCycle.fmu",
        state_variables=Battery_state_vars,
        input_vec=['I_req'],
        output_vec=['SoC',
                    'theveninBasedBattery.SoH'],
        horizon_num=5,
        points_per_sec=.1
    )

    test_df = pd.read_csv('plots_data/test_cycle.csv')
    test_df = test_df.rename(columns={'x': 'Time'})
    test_df['y'] = test_df['y']*C_rate_per_second*1.075/1.1
    test_df['Time'] = test_df['Time'].apply(lambda x: max(x, 0))
    test_df['Time'] = pd.to_datetime(test_df['Time'], unit='m')
    test_df = test_df.set_index('Time').resample("s").mean().interpolate(
        method='time', axis=0)

    test_df['Time'] = test_df.reset_index()['Time'].values.astype('int')/10**9
    test_df = test_df.set_index('Time')

    start_time = 0
    final_time = 10140

    states = mpc_optimizer.simulate(start_time, final_time, test_df, save_all=True)
    states[State_vars_aliases_dict['U']] *= -1

    states = states.rename(columns={v: k for k, v in State_vars_aliases_dict.items()})
    var, soc, t = states['U'], states['SoC'], states['time']

    st = np.argmin(np.abs(t - 6500)) if is_charging else 0
    en = np.argmin(np.abs(t - 10000)
                   ) if is_charging else np.argmin(np.abs(t - 3800))
    time = t[st:en]/60-110 if is_charging else t[st:en]/60
    simulation_res = pd.DataFrame({"time": time, "voltage": var[st:en]})
    simulation_res['type'] = 'simulation'
    modifier = ("" if is_charging else "dis") + "charging"
    measurement_output_df = pd.read_csv(
        'plots_data/measurement_' + modifier + '.csv').rename(columns={'x': 'time', 'y': 'voltage'})
    measurement_output_df['type'] = 'measurement'

    return alt.Chart(simulation_res.append(measurement_output_df)).mark_line().encode(
        x=alt.X('time:Q', title='time(min)'),
        y=alt.Y('voltage:Q', scale=alt.Scale(zero=False), title='U_bat(V)'),
        color=alt.Color('type:N', scale=alt.Scale(scheme='set1'))
    )

In [70]:
test_df = pd.read_csv('plots_data/test_cycle.csv')
test_df = test_df.rename(columns={'x': 'Time'})
test_df['y'] = test_df['y']*1.075
test_df['Time'] = test_df['Time'].apply(lambda x: max(x, 0))
test_df['Time'] = pd.to_datetime(test_df['Time'], unit='m')
test_df = test_df.set_index('Time').resample("s").mean().interpolate(
    method='time', axis=0)

test_df['Time'] = test_df.reset_index()['Time'].values.astype('int')/10**9/60
test_df = test_df.set_index('Time')

alt.Chart(test_df.reset_index()).mark_line(color='red').encode(
    x=alt.X('Time:Q', title='time(min)'),
    y=alt.Y('y:Q', title='I_req(A)'),
).configure_view(
    height=300,
    width=500
).configure_legend(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=17,
    titleFontSize=19,
).configure_axis(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=17,
    titleFontSize=19,
)

alt.Chart(...)

In [38]:
alt.vconcat(
    get_test_voltage_response(False).properties(
        title='charging'
    ),
    get_test_voltage_response(True).properties(
        title='discharging')
).configure_view(
    height=300,
    width=600
).configure_legend(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=17,
    titleFontSize=19,
).configure_axis(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=17,
    titleFontSize=19,
)

100%|██████████| 1014/1014 [00:01<00:00, 722.84it/s]


alt.VConcatChart(...)